如果要加入新資料
1. NewDataLoader(BasicLoader)
2. LoaderMapping新增欄位
3. .yml新增欄位

In [1]:
### reload magic
%reload_ext autoreload
%autoreload 2

import sys
import yaml
import torch
import torchvision
import torch.utils.cpp_extension

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping, DeviceStatsMonitor

from src.data_manager import DataManager
from src.model_director import ModelDirector

# os.environ['CUDA_VISIBLE_DEVICES']="0"
print('Python version', sys.version)
print('CUDA_HOME', torch.utils.cpp_extension.CUDA_HOME)
print('CudaToolKit Version', torch.version.cuda)
print('Cudnn Version', torch.backends.cudnn.version())
print('torch Version', torch.__version__)
print('torchvision Version', torchvision.__version__)

config_file = './exp_config/gru_adv_config_large.yml'
with open(config_file, "r") as content:
    config = yaml.safe_load(content) 

Python version 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
CUDA_HOME /usr/local/cuda
CudaToolKit Version 11.8
Cudnn Version 8700
torch Version 2.2.1+cu118
torchvision Version 0.17.1+cu118


In [2]:
dm = DataManager(
    data_meta_info=config['train_config']['data_meta_info'], 
    **config['train_config']['data_loader_params']
)

100%|██████████| 362024/362024 [00:02<00:00, 169242.57it/s]


RainLoaderJay: time list is loaded.


100%|██████████| 366871/366871 [00:02<00:00, 168761.46it/s]


RadarLoaderJay: time list is loaded.
[DataManager] Total data collected: 348797, Sampling Rate: 5
[DataManager] Training Data Size: 41856, Validating Data Size: 13951, Testing Data Size: 13951 
[DataManager] Image Shape: (540, 420), Batch Size: 64


In [3]:
model = ModelDirector(
    data_info = dm.get_data_info(),
    **config['model'],
).build_model()

[EncoderParams] InputChannelCount: 2, Shape: (540, 420)
[ForecasterPONI] TargetLen: 3, TeacherForcing: 0.5, PoniInputChannel: 1
[Discriminator] Downsample: 5
[ModelDirector] Using CAPN_PONI model


In [4]:
logger = TensorBoardLogger(save_dir='logs', name=config['model']['model_config']['name'])
checkpoint_callback = model.get_checkpoint_callback()
trainer = Trainer(
    benchmark=True,
    accelerator='gpu',
    devices=[0],
    fast_dev_run=False, # True: debug and compile once 
    logger=logger,
    check_val_every_n_epoch=1,
    max_epochs=50, 
    callbacks=[
        DeviceStatsMonitor(cpu_stats=True),
        EarlyStopping(monitor="val_loss", patience=10), 
        checkpoint_callback
    ]
)
trainer.fit(model, dm)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: logs/BalancedGRUAdvPONI
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type            | Params
--------------------------------------------------
0 | encoder       | Encoder         | 3.7 M 
1 | forecaster    | ForecasterPONI  | 3.9 M 
2 | discriminator | Discriminator   | 1.2 M 
3 | loss_fn       | WeightedMaeLoss | 0     
4 | dis_loss_fn   | BCELoss         | 0     
--------------------------------------------------
8.7 M     Trainable params
0         Non-trainable params
8.7 M     Total params
34.907    Total estimated model params size (MB)


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:09<00:00,  4.96s/it]

/home/dong1128/deepQPF_research/src/utils/performance_diagram.py:199: RuntimeWarning: invalid value encountered in scalar divide
  output.append(2 * pd * sr / (pd + sr))


Epoch 0: 100%|██████████| 654/654 [1:28:48<00:00,  8.15s/it, v_num=0, loss_P_step=1.280, loss_GD_step=3.330, loss_G_step=1.300, loss_D_step=0.00524, loss_P_epoch=1.410, loss_GD_epoch=3.610, loss_G_epoch=1.430, loss_D_epoch=0.00828]

Epoch 0, global step 1308: 'val_loss' reached 1.28882 (best 1.28882), saving model to '/home/dong1128/deepQPF_research/checkpoint/epoch=0_val_loss=1.288815.ckpt' as top 1


Epoch 1:  15%|█▍        | 97/654 [12:49<1:13:37,  7.93s/it, v_num=0, loss_P_step=1.100, loss_GD_step=2.430, loss_G_step=1.110, loss_D_step=0.00357, loss_P_epoch=1.410, loss_GD_epoch=3.610, loss_G_epoch=1.430, loss_D_epoch=0.00828] 

/home/dong1128/miniconda3/envs/deepQPF_research/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
